In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [ ]:
import sys 
sys.path.append("..")

In [ ]:
import numpy as np
import copy
from tqdm  import tqdm

In [ ]:
import torch as ch
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import ImageNet
from robustness.tools import helpers

In [ ]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import quant_modules

In [ ]:
device = ch.device("cuda" if (ch.cuda.is_available()) else "cpu")

In [ ]:
ch.cuda.empty_cache()

# Make dataset and loaders

In [ ]:
NUM_WORKERS = 4
BATCH_SIZE = 64

In [ ]:
# Hard-coded dataset, architecture, batch size, workers
ds = ImageNet('../../datasets/ImageNet')

In [ ]:
train_loader, val_loader = ds.make_loaders(batch_size=BATCH_SIZE, workers=NUM_WORKERS)

# Create the quantized model and calibrate it with the training dataset

# load Model

In [ ]:
q_model = models.resnet50(pretrained=True)

In [ ]:
q_model.load_state_dict(ch.load('../../weights/q_imagenet_linf_8.pt'), strict=False)

In [ ]:
quant_modules.deactivate()

In [ ]:
m, checkpoint = model_utils.make_and_restore_model(parallel= True, arch='resnet50', dataset=ds, resume_path= '../../weights/imagenet_linf_8.pt')
fp_model = models.resnet50(pretrained=True)
fp_model.load_state_dict(m.model.state_dict())

In [ ]:
q_model = q_model.to(device)
fp_model = fp_model.to(device)
fp_model.eval()
q_model.eval()

In [ ]:
normalizer = helpers.InputNormalize(ch.tensor([0.485, 0.456, 0.406]).to(device), ch.tensor([0.229, 0.224, 0.225]).to(device))

# Test Clean Accuracy

In [ ]:
iterator = tqdm(enumerate(val_loader), total=len(val_loader))
total = 0
q_correct = 0
fp_correct = 0
for i, (inp, target) in iterator:
    target = target.cuda(non_blocking=True)
    inp = normalizer(inp.cuda())
    
    q_logits = q_model(inp)
    _, q_pred = q_logits.topk(1, 1, True, True)
    q_pred = q_pred.t()[0]
    q_correct += (q_pred == target).sum().cpu().numpy()
    
    
    fp_logits = fp_model(inp)
    _, fp_pred = fp_logits.topk(1, 1, True, True)
    fp_pred = fp_pred.t()[0]
    fp_correct += (fp_pred == target).sum().cpu().numpy()
    
    total += BATCH_SIZE
    ch.cuda.empty_cache()

In [ ]:
print('fp clean accuracy: ' + str(fp_correct/total) + '  clean accuracy: ' + str(q_correct/total))

# FGSM

In [ ]:
# FGSM attack code
def fgsm_attack(image, orig_img, step, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    adv_image = image + step*sign_data_grad
    
    # Adding clipping to maintain [0,1] range
    A = ch.clamp(adv_image - orig_img, -epsilon, epsilon)
    perturbed_image = ch.clamp(orig_img + A, 0, 1)

    # Return the perturbed image
    return perturbed_image

# PGD

In [ ]:
criterion = ch.nn.CrossEntropyLoss()
def PGD( model, fp_model, device, test_loader, step, epsilon, grad_iterations):
    
    model.eval()
    fp_model.eval()
    # Accuracy counter
    success = 0
    Agree = 0
    q_wrong = 0
    fp_correct = 0
    Q_success = 0

    # Loop over all examples in test set
    iterator = tqdm(enumerate(test_loader), total=len(test_loader))
    for i, (data, target) in  iterator:
        
        # Send the data and label to the device
        # data = ch.clamp(data + 2 * (ch.rand_like(data) - 0.5) * epsilon, 0, 1)
        data, target = data.to(device), target.to(device)

        # If the initial prediction is wrong, dont bother attacking, just move on
        q_pred = model(normalizer(data).cuda()).max(1, keepdim=True)[1].t()[0] # get the index of the max log-probability
        fp_pred = fp_model(normalizer(data)).max(1, keepdim=True)[1].t()[0] # get the index of the max log-probability
        
        index = ch.logical_and((target == q_pred), (target == fp_pred))
        Q_success += ch.sum(ch.logical_not(target == q_pred))
        data = data[index]
        target = target[index]
        orig_cpy = data.clone().detach()
        Agree += len(target)
        
        
        for iters in range(0,grad_iterations):
            
            if len(target) == 0:
                continue
            
            data = data.clone().detach().requires_grad_(True)
            
            output = model(normalizer(data))
            loss = criterion(output,target)
            
            # Zero all existing gradients
            model.zero_grad()
            
            # Calculate gradients of model in backward pass
            loss.backward()
            
            # Collect datagrad
            data_grad = data.grad.data
            
            # Call FGSM Attack
            perturbed_data = fgsm_attack(data, orig_cpy, step, epsilon, data_grad)

            # Re-classify the perturbed image
            q_pred = model(normalizer(perturbed_data)).max(1, keepdim=True)[1].t()[0]
            fp_pred = fp_model(normalizer(perturbed_data)).max(1, keepdim=True)[1].t()[0]

            # Check for success
            q_w = ch.logical_not(target == q_pred)
            fp_c = (target == fp_pred)
            
            fp_c_q_w = ch.logical_and(q_w, fp_c)
            index = ch.logical_not(fp_c_q_w)
            
            data = perturbed_data[index]
            orig_cpy = orig_cpy[index]
            
            success += ch.sum(fp_c_q_w)
            q_wrong += ch.sum(fp_c_q_w)
            Q_success += ch.sum(fp_c_q_w)
            fp_correct += ch.sum(fp_c_q_w)
            
            target = target[index]
            
            if iters == (grad_iterations -1) or len(target) == 0:
                q_wrong += ch.sum(ch.logical_and(q_w, index))
                fp_correct += ch.sum(ch.logical_and(fp_c, index))
                Q_success += ch.sum(ch.logical_and(q_w, index))

        ch.cuda.empty_cache()
        # Calculate final accuracy for this epsilon
        if i%100 == 0:
            print("Total: {} \t Success: {} \t Q_W:{} \t FP_W:{} \t Robust_acc: {:.2f} ".format(Agree, success, q_wrong ,Agree - fp_correct, 100* (1- Q_success/((i+1) *BATCH_SIZE)) ))
    print("Total: {} \t Success: {} \t Q_W:{} \t FP_W:{} \t Robust_acc: {:.2f} ".format(Agree, success, q_wrong ,Agree - fp_correct, 100* (1- Q_success/((i+1) *BATCH_SIZE)) ))

In [ ]:
PGD(q_model, fp_model, device, val_loader, 0.00375, 0.03, 20)

# DIVA

In [ ]:
diva_criterion = ch.nn.Softmax()
def DIVA(q_model, fp_model, c, device, test_loader, step, epsilon, grad_iterations):
    
    q_model.eval()
    fp_model.eval()
    # Accuracy counter
    success = 0
    Agree = 0
    q_wrong = 0
    fp_correct = 0
    Q_success = 0

    # Loop over all examples in test set
    iterator = tqdm(enumerate(test_loader), total=len(test_loader))
    for i, (data, target) in  iterator:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # If the initial prediction is wrong, dont bother attacking, just move on
        q_pred = q_model(normalizer(data)).max(1, keepdim=True)[1].t()[0] # get the index of the max log-probability
        fp_pred = fp_model(normalizer(data)).max(1, keepdim=True)[1].t()[0] # get the index of the max log-probability
        
        index = ch.logical_and((target == q_pred), (target == fp_pred))
        Q_success += ch.sum(ch.logical_not(target == q_pred))
        data = data[index]
        target = target[index]
        orig_cpy = data.clone().detach()
        Agree += len(target)
        
        for iters in range(0,grad_iterations):
            
            if len(target) == 0:
                continue
            
            data = data.clone().detach().requires_grad_(True)
            
            output1 = q_model(normalizer(data))
            output2 = fp_model(normalizer(data))
            labels = [target.tolist()]
            loss1 = ch.mean(diva_criterion(output1)[[i for i in range(0,output1.shape[0])],labels ][0])
            loss2 = ch.mean(diva_criterion(output2)[[i for i in range(0,output2.shape[0])],labels ][0])

    
            loss = loss2 - c*loss1
            
            # Zero all existing gradients
            q_model.zero_grad()
            fp_model.zero_grad()
            
            # Calculate gradients of model in backward pass
            loss.backward()
            
            # Collect datagrad
            data_grad = data.grad.data
            
            # Call FGSM Attack
            perturbed_data = fgsm_attack(data, orig_cpy, step, epsilon, data_grad)

            # Re-classify the perturbed image
            q_pred = q_model(normalizer(perturbed_data)).max(1, keepdim=True)[1].t()[0]
            fp_pred = fp_model(normalizer(perturbed_data)).max(1, keepdim=True)[1].t()[0]

            # Check for success
            q_w = ch.logical_not(target == q_pred)
            fp_c = (target == fp_pred)
            
            fp_c_q_w = ch.logical_and(q_w, fp_c)
            index = ch.logical_not(fp_c_q_w)
            
            data = perturbed_data[index]
            orig_cpy = orig_cpy[index]
            
            success += ch.sum(fp_c_q_w)
            q_wrong += ch.sum(fp_c_q_w)
            Q_success += ch.sum(fp_c_q_w)
            fp_correct += ch.sum(fp_c_q_w)
            
            target = target[index]
            
            if iters == (grad_iterations -1) or len(target) == 0:
                q_wrong += ch.sum(ch.logical_and(q_w, index))
                Q_success += ch.sum(ch.logical_and(q_w, index))
                fp_correct += ch.sum(ch.logical_and(fp_c, index))

        ch.cuda.empty_cache()
        # Calculate final accuracy for this epsilon
        if i%100 == 0:
            print("Total: {} \t Success: {} \t Q_W:{} \t FP_W:{} \t Adv_acc: {:.2f} ".format(Agree, success, q_wrong ,Agree - fp_correct, 100* (1- Q_success/((i+1) *BATCH_SIZE)) ))
    print("Total: {} \t Success: {} \t Q_W:{} \t FP_W:{} \t Robust_acc: {:.2f} ".format(Agree, success, q_wrong ,Agree - fp_correct, 100* (1- Q_success/((i+1) *BATCH_SIZE)) ))

In [ ]:
DIVA(q_model, fp_model, 1.5, device, val_loader, 0.00375, 0.03, 20)